# Demo - Extendre ChatGPT amb LangChain

In [1]:
import os

from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import VectorDBQA
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.agents import initialize_agent, Tool
from langchain.text_splitter import CharacterTextSplitter
from langchain import GoogleSearchAPIWrapper, PromptTemplate, LLMChain
from langchain.document_loaders import TextLoader, WebBaseLoader

from dotenv import load_dotenv

load_dotenv();

In [2]:
# Global vars
embeddings = OpenAIEmbeddings()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

### Crear i emmagatzemar a disc els "embeddings"


In [3]:
URL_NOTICIA="https://www.diaridegirona.cat/girona/2023/03/06/vilablareix-comunitat-plaques-fotovoltaiques-84140595.html"

# docs = text_splitter.split_documents(WebBaseLoader(URL_NOTICIA).load())

# noticia_vectordb = FAISS.from_documents(docs, embeddings)
# noticia_vectordb.save_local("faiss_data/diaridegirona_website")

### Carregar de disc els "embeddings"

In [4]:
noticia_vectordb = FAISS.load_local("faiss_data/diaridegirona_website", embeddings)

### ChatGPT prompt - Preguntes i respostes sobre l'article del Diari de Girona

In [5]:
noticia = VectorDBQA.from_chain_type(llm=ChatOpenAI(temperature=0.7), chain_type="stuff", vectorstore=noticia_vectordb)

In [6]:
noticia.run("Fes un resum de la notícia en una frase")

"L'Ajuntament de Vilablareix instal·larà plaques solars fotovoltaiques a la coberta del pavelló municipal per subministrar energia elèctrica als edificis municipals i als veïns que en vulguin formar part."

In [7]:
noticia.run("Com es diu la regidora de medi ambient de l'ajuntament de Vilablareix?")

"La regidora de medi ambient de l'ajuntament de Vilablareix es diu Imma Vilà. Ho podeu llegir a l'article: «Formaran part del projecte d’autoconsum compartit els equipaments municipals i llars de veïns i veïnes del poble. Es prioritzaran les llars més vulnerables» ha explicat la regidora de medi ambient, Imma Vilà."

In [8]:
noticia.run("Quina és la inversió inicial per a la instal.lació de plaques solars a Vilablareix?")

'La inversió inicial per a la instal·lació de plaques fotovoltaiques a Vilablareix és de 48.878,05 euros per una instal·lació de 58,65 kWp, i està subvencionada per la Diputació de Girona amb 36.000,00 euros.'

### Chat amb "memòria" i cerca a Google per complementar respostes

In [45]:
tools = [
    Tool(
        name = "Google Search",
        func=GoogleSearchAPIWrapper().run,
        description="Cerca a Google per complementar informació"
    ),
]

memory = ConversationBufferMemory(memory_key="chat_history")

agent_chain = initialize_agent(tools, ChatOpenAI(temperature=0.7), agent="conversational-react-description", verbose=True, max_iterations=10, memory=memory)


In [46]:
agent_chain.run(input="Quin es el número de habitants de Vilablareix, Girona?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Google Search
Action Input: Number of inhabitants in Vilablareix, Girona
Observation: Vilablareix is a small village in the province of Girona and autonomous community of Catalonia, Spain. Vilablareix. Municipality. Restaurant Mas Marroch, Vilablareix: See 103 unbiased reviews of Restaurant Mas Marroch, rated 4.5 of 5 on Tripadvisor and ranked #1 of 6 restaurants in ... 44 listings of houses and flats for sale in Vilablareix, Spain, from 205000 euros. Houses and flats for sale in Vilablareix from private owners and real ... This review is the subjective opinion of a Tripadvisor member and not of Tripadvisor LLC. Tripadvisor performs checks on reviews. The site will have a health park for the residents of Vilablareix and the surrounding area, who will also be able to get there by bicycle thanks to the ... König has been collaborating for many years with the Astrid Foundation through the employment inte

'According to available data, the population of Vilablareix, Girona is around 2,194 residents, which is about 0% of the population of the province.'

In [47]:
agent_chain.run(input="Assumint una ocupació de 2.5 habitants per llar. Quin porcentatge de habitants implicaria 160 llars?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Assuming an occupancy of 2.5 residents per household, 160 households would house approximately 400 residents. To find the percentage of the total population this represents, we would need to know the total population of the area in question.

> Finished chain.


'Assuming an occupancy of 2.5 residents per household, 160 households would house approximately 400 residents. To find the percentage of the total population this represents, we would need to know the total population of the area in question.'

In [48]:
agent_chain.run(input="Fes el càlcul amb la població total de Vilablareix, Girona")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Based on the population of Vilablareix, Girona being around 2,194 residents, 160 households with an assumed occupancy of 2.5 residents per household would house approximately 400 residents. This represents around 18.2% of the total population of Vilablareix, Girona.

> Finished chain.


'Based on the population of Vilablareix, Girona being around 2,194 residents, 160 households with an assumed occupancy of 2.5 residents per household would house approximately 400 residents. This represents around 18.2% of the total population of Vilablareix, Girona.'